# Introduction

In this guide, we will walk you through building a powerful semantic search engine using [Couchbase](https://www.couchbase.com) as the backend database and [CrewAI](https://github.com/crewAIInc/crewAI) for agent-based RAG operations. CrewAI allows us to create specialized agents that can work together to handle different aspects of the RAG workflow, from document retrieval to response generation. This tutorial is designed to be beginner-friendly, with clear, step-by-step instructions that will equip you with the knowledge to create a fully functional semantic search system from scratch.

How to run this tutorial
----------------------
This tutorial is available as a Jupyter Notebook (.ipynb file) that you can run
interactively. You can access the original notebook here.

You can either:
- Download the notebook file and run it on [Google Colab](https://colab.research.google.com)
- Run it on your system by setting up the Python environment

Before you start
---------------

1. Create and Deploy Your Free Tier Operational cluster on [Capella](https://cloud.couchbase.com/sign-up)
   - To get started with [Couchbase Capella](https://cloud.couchbase.com), create an account and use it to deploy
     a forever free tier operational cluster
   - This account provides you with an environment where you can explore and learn
     about Capella with no time constraint
   - To learn more, please follow the [Getting Started Guide](https://docs.couchbase.com/cloud/get-started/create-account.html)

2. Couchbase Capella Configuration
   When running Couchbase using Capella, the following prerequisites need to be met:
   - Create the database credentials to access the required bucket (Read and Write) used in the application
   - Allow access to the Cluster from the IP on which the application is running by following the [Network Security documentation](https://docs.couchbase.com/cloud/security/security.html#public-access)

# Setting the Stage: Installing Necessary Libraries

We'll install the following key libraries:
- `datasets`: For loading and managing our training data
- `langchain-couchbase`: To integrate Couchbase with LangChain for vector storage and caching
- `langchain-openai`: For accessing OpenAI's embedding and chat models
- `crewai`: To create and orchestrate our AI agents for RAG operations
- `python-dotenv`: For securely managing environment variables and API keys

These libraries provide the foundation for building a semantic search engine with vector embeddings,
database integration, and agent-based RAG capabilities.

In [3]:
%pip install --quiet datasets==3.5.0 langchain-couchbase==0.3.0 langchain-openai==0.3.13 crewai==0.114.0 python-dotenv==1.1.0

In [1]:
%pip install langchain-community litellm langchain-google-genai

# Importing Necessary Libraries
The script starts by importing a series of libraries required for various tasks, including handling JSON, logging, time tracking, Couchbase connections, embedding generation, and dataset loading.

In [2]:
import getpass
import json
import logging
import os
import time
from datetime import timedelta

from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.diagnostics import PingState, ServiceType
from couchbase.exceptions import (InternalServerFailureException,
                                  QueryIndexAlreadyExistsException,
                                  ServiceUnavailableException)
from couchbase.management.buckets import CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from datasets import load_dataset
from dotenv import load_dotenv
from crewai.tools import tool
from langchain_couchbase.vectorstores import CouchbaseSearchVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from crewai import Agent, Crew, Process, Task, LLM

# Setup Logging
Logging is configured to track the progress of the script and capture any errors or warnings.

In [3]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Suppress httpx logging
logging.getLogger('httpx').setLevel(logging.CRITICAL)

# Loading Sensitive Information
In this section, we prompt the user to input essential configuration settings needed. These settings include sensitive information like database credentials, and specific configuration names. Instead of hardcoding these details into the script, we request the user to provide them at runtime, ensuring flexibility and security.

The script uses environment variables to store sensitive information, enhancing the overall security and maintainability of your code by avoiding hardcoded values.

In [2]:
# Load environment variables
load_dotenv()

# Configuration
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or input("Enter your OpenAI API key: ")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set")

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY') or input("Enter your GEMINI AI Studio API key: ")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY is not set")

CB_HOST = os.getenv('CB_HOST') or input("Enter Couchbase host (default: couchbase://localhost): ") or 'couchbase://localhost'
CB_USERNAME = os.getenv('CB_USERNAME') or input("Enter Couchbase username (default: Administrator): ") or 'Administrator'
CB_PASSWORD = os.getenv('CB_PASSWORD') or getpass.getpass("Enter Couchbase password (default: password): ") or 'password'
CB_BUCKET_NAME = os.getenv('CB_BUCKET_NAME') or input("Enter bucket name (default: vector-search-testing): ") or 'vector-search-testing'
INDEX_NAME = os.getenv('INDEX_NAME') or input("Enter index name (default: vector_search_crew): ") or 'vector_search_crew'
SCOPE_NAME = os.getenv('SCOPE_NAME') or input("Enter scope name (default: shared): ") or 'shared'
COLLECTION_NAME = os.getenv('COLLECTION_NAME') or input("Enter collection name (default: crew): ") or 'crew'

print("Configuration loaded successfully")

NameError: name 'load_dotenv' is not defined

In [1]:
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY') or input("Enter your GEMINI AI Studio API key: ")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY is not set")



NameError: name 'os' is not defined

# Connecting to the Couchbase Cluster
Connecting to a Couchbase cluster is the foundation of our project. Couchbase will serve as our primary data store, handling all the storage and retrieval operations required for our semantic search engine. By establishing this connection, we enable our application to interact with the database, allowing us to perform operations such as storing embeddings, querying data, and managing collections. This connection is the gateway through which all data will flow, so ensuring it's set up correctly is paramount.

In [6]:
# Connect to Couchbase
try:
    auth = PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)
    options = ClusterOptions(auth)
    cluster = Cluster(CB_HOST, options)
    cluster.wait_until_ready(timedelta(seconds=5))
    print("Successfully connected to Couchbase")
except Exception as e:
    print(f"Failed to connect to Couchbase: {str(e)}")
    raise

Successfully connected to Couchbase


# Verifying Search Service Availability
 In this section, we verify that the Couchbase Search (FTS) service is available and responding correctly. This is a crucial check because our vector search functionality depends on it. If any issues are detected with the Search service, the function will raise an exception, allowing us to catch and handle problems early before attempting vector operations.


In [7]:
def check_search_service(cluster):
    """Verify search service availability using ping"""
    try:
        # Get ping result
        ping_result = cluster.ping()
        search_available = False

        # Check if search service is responding
        for service_type, endpoints in ping_result.endpoints.items():
            if service_type == ServiceType.Search:
                for endpoint in endpoints:
                    if endpoint.state == PingState.OK:
                        search_available = True
                        print(f"Search service is responding at: {endpoint.remote}")
                        break
                break

        if not search_available:
            raise RuntimeError("Search/FTS service not found or not responding")

        print("Search service check passed successfully")
    except Exception as e:
        print(f"Health check failed: {str(e)}")
        raise
try:
    check_search_service(cluster)
except Exception as e:
    print(f"Failed to check search service: {str(e)}")
    raise

Search service is responding at: 3.133.133.255:18094
Search service check passed successfully


## Setting Up Collections in Couchbase

The setup_collection() function handles creating and configuring the hierarchical data organization in Couchbase:

1. Bucket Creation:
   - Checks if specified bucket exists, creates it if not
   - Sets bucket properties like RAM quota (1024MB) and replication (disabled)
   - Note: If you are using Capella, create a bucket manually called vector-search-testing(or any name you prefer) with the same properties.

2. Scope Management:  
   - Verifies if requested scope exists within bucket
   - Creates new scope if needed (unless it's the default "_default" scope)

3. Collection Setup:
   - Checks for collection existence within scope
   - Creates collection if it doesn't exist
   - Waits 2 seconds for collection to be ready

Additional Tasks:
- Creates primary index on collection for query performance
- Clears any existing documents for clean state
- Implements comprehensive error handling and logging

The function is called twice to set up:
1. Main collection for vector embeddings
2. Cache collection for storing results


In [8]:
def setup_collection(cluster, bucket_name, scope_name, collection_name):
    try:
        # Check if bucket exists, create if it doesn't
        try:
            bucket = cluster.bucket(bucket_name)
            logging.info(f"Bucket '{bucket_name}' exists.")
        except Exception as e:
            logging.info(f"Bucket '{bucket_name}' does not exist. Creating it...")
            bucket_settings = CreateBucketSettings(
                name=bucket_name,
                bucket_type='couchbase',
                ram_quota_mb=1024,
                flush_enabled=True,
                num_replicas=0
            )
            cluster.buckets().create_bucket(bucket_settings)
            time.sleep(2)  # Wait for bucket creation to complete and become available
            bucket = cluster.bucket(bucket_name)
            logging.info(f"Bucket '{bucket_name}' created successfully.")

        bucket_manager = bucket.collections()

        # Check if scope exists, create if it doesn't
        scopes = bucket_manager.get_all_scopes()
        scope_exists = any(scope.name == scope_name for scope in scopes)

        if not scope_exists and scope_name != "_default":
            logging.info(f"Scope '{scope_name}' does not exist. Creating it...")
            bucket_manager.create_scope(scope_name)
            logging.info(f"Scope '{scope_name}' created successfully.")

        # Check if collection exists, create if it doesn't
        collections = bucket_manager.get_all_scopes()
        collection_exists = any(
            scope.name == scope_name and collection_name in [col.name for col in scope.collections]
            for scope in collections
        )

        if not collection_exists:
            logging.info(f"Collection '{collection_name}' does not exist. Creating it...")
            bucket_manager.create_collection(scope_name, collection_name)
            logging.info(f"Collection '{collection_name}' created successfully.")
        else:
            logging.info(f"Collection '{collection_name}' already exists. Skipping creation.")

        # Wait for collection to be ready
        collection = bucket.scope(scope_name).collection(collection_name)
        time.sleep(2)  # Give the collection time to be ready for queries

        # Ensure primary index exists
        try:
            cluster.query(f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{bucket_name}`.`{scope_name}`.`{collection_name}`").execute()
            logging.info("Primary index present or created successfully.")
        except Exception as e:
            logging.warning(f"Error creating primary index: {str(e)}")

        # Clear all documents in the collection
        try:
            query = f"DELETE FROM `{bucket_name}`.`{scope_name}`.`{collection_name}`"
            cluster.query(query).execute()
            logging.info("All documents cleared from the collection.")
        except Exception as e:
            logging.warning(f"Error while clearing documents: {str(e)}. The collection might be empty.")

        return collection
    except Exception as e:
        raise RuntimeError(f"Error setting up collection: {str(e)}")

setup_collection(cluster, CB_BUCKET_NAME, SCOPE_NAME, COLLECTION_NAME)


# Configuring and Initializing Couchbase Vector Search Index for Semantic Document Retrieval

Semantic search requires an efficient way to retrieve relevant documents based on a user's query. This is where the Couchbase Vector Search Index comes into play. In this step, we load the Vector Search Index definition from a JSON file, which specifies how the index should be structured. This includes the fields to be indexed, the dimensions of the vectors, and other parameters that determine how the search engine processes queries based on vector similarity.

This CrewAI vector search index configuration requires specific default settings to function properly. This tutorial uses the bucket named `vector-search-testing` with the scope `shared` and collection `crew`. The configuration is set up for vectors with exactly `1536 dimensions`, using `dot product` similarity and optimized for `recall`. If you want to use a different bucket, scope, or collection, you will need to modify the index configuration accordingly.

For more information on creating a vector search index, please follow the instructions at [Couchbase Vector Search Documentation](https://docs.couchbase.com/cloud/vector-search/create-vector-search-index-ui.html).

In [29]:
# Load index definition
try:
    with open('crew_index.json', 'r') as file:
        index_definition = json.load(file)
except FileNotFoundError as e:
    print(f"Error: crew_index.json file not found: {str(e)}")
    raise
except json.JSONDecodeError as e:
    print(f"Error: Invalid JSON in crew_index.json: {str(e)}")
    raise
except Exception as e:
    print(f"Error loading index definition: {str(e)}")
    raise

# Creating or Updating Search Indexes

With the index definition loaded, the next step is to create or update the **Vector Search Index** in Couchbase. This step is crucial because it optimizes our database for vector similarity search operations, allowing us to perform searches based on the semantic content of documents rather than just keywords. By creating or updating a Vector Search Index, we enable our search engine to handle complex queries that involve finding semantically similar documents using vector embeddings, which is essential for a robust semantic search engine.

In [30]:
try:
    scope_index_manager = cluster.bucket(CB_BUCKET_NAME).scope(SCOPE_NAME).search_indexes()

    # Check if index already exists
    existing_indexes = scope_index_manager.get_all_indexes()
    index_name = index_definition["name"]

    if index_name in [index.name for index in existing_indexes]:
        logging.info(f"Index '{index_name}' found")
    else:
        logging.info(f"Creating new index '{index_name}'...")

    # Create SearchIndex object from JSON definition
    search_index = SearchIndex.from_json(index_definition)

    # Upsert the index (create if not exists, update if exists)
    scope_index_manager.upsert_index(search_index)
    logging.info(f"Index '{index_name}' successfully created/updated.")

except QueryIndexAlreadyExistsException:
    logging.info(f"Index '{index_name}' already exists. Skipping creation/update.")
except ServiceUnavailableException:
    raise RuntimeError("Search service is not available. Please ensure the Search service is enabled in your Couchbase cluster.")
except InternalServerFailureException as e:
    logging.error(f"Internal server error: {str(e)}")
    raise

## Setting Up AI Components

This section initializes two key components needed for our RAG system:

1.  Embeddings:
   - Converts text into high-dimensional vector representations (embeddings)
   - These embeddings enable semantic search by capturing the meaning of text
   - Required for vector similarity search in Couchbase

2.  Language Model:
   - Temperature set to 0.2 for balanced creativity and focus
   - Serves as the cognitive engine for CrewAI agents
   - Powers agent reasoning, decision-making, and task execution
   - Enables agents to:
     - Process and understand retrieved context from vector search
     - Generate thoughtful responses based on that context
     - Follow instructions defined in agent roles and goals
     - Collaborate with other agents in the crew
   - The relatively low temperature (0.2) ensures agents produce reliable,
     consistent outputs while maintaining some creative problem-solving ability

In the CrewAI framework, the LLM acts as the "brain" for each agent, allowing them
to interpret tasks, retrieve relevant information via the RAG system, and generate
appropriate outputs based on their specialized roles and expertise.

In [31]:
from typing import Any, Dict, List, Optional
import os
from openai import OpenAI
from langchain_core.embeddings import Embeddings


class NebiusE5MistralEmbeddings(Embeddings):
    """Nebius E5 Mistral 7B embedding model integration for LangChain.

    Key init args:
        model: str
            Name of Nebius model to use, defaults to "intfloat/e5-mistral-7b-instruct"
        api_key: Optional[str]
            API key for Nebius. If not provided, will look for NEBIUS_API_KEY env var.
        base_url: str
            Base URL for Nebius API, defaults to "https://api.studio.nebius.com/v1/"
        client: Optional[OpenAI]
            Pre-configured OpenAI client. If provided, other connection params are ignored.

    Instantiate:
        .. code-block:: python

            from nebius_embeddings import NebiusE5MistralEmbeddings

            # Basic usage with API key in environment variable
            embed = NebiusE5MistralEmbeddings()

            # Or with explicit parameters
            embed = NebiusE5MistralEmbeddings(
                model="intfloat/e5-mistral-7b-instruct",
                api_key="your-api-key"
            )

    Embed single text:
        .. code-block:: python

            input_text = "The meaning of life is 42"
            embedding = embed.embed_query(input_text)
            # Returns a list of floats representing the embedding

    Embed multiple texts:
        .. code-block:: python

            input_texts = ["Document 1...", "Document 2..."]
            embeddings = embed.embed_documents(input_texts)
            # Returns a list of lists of floats, each representing a document embedding
    """

    def __init__(
        self,
        model: str = "intfloat/e5-mistral-7b-instruct",
        api_key: Optional[str] = None,
        base_url: str = "https://api.studio.nebius.com/v1/",
        client: Optional[Any] = None,
        **kwargs: Any,
    ):
        """Initialize the Nebius E5 Mistral embeddings.

        Args:
            model: The model to use for embeddings
            api_key: The API key to use. If not provided, will look for NEBIUS_API_KEY env var
            base_url: The base URL for the Nebius API
            client: Pre-configured OpenAI client. If provided, other connection params are ignored
            **kwargs: Additional parameters to pass to the client
        """
        self.model = model

        if client is not None:
            self.client = client
        else:
            api_key = api_key or os.environ.get("NEBIUS_API_KEY")
            if not api_key:
                raise ValueError(
                    "Nebius API key must be provided as an argument or set as environment "
                    "variable NEBIUS_API_KEY"
                )

            self.client = OpenAI(
                base_url=base_url,
                api_key=api_key,
                **kwargs,
            )

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed a list of documents using the Nebius API.

        Args:
            texts: The list of texts to embed

        Returns:
            List of embeddings, one for each text
        """
        # Handle empty inputs
        if not texts:
            return []

        # Process in batch
        embeddings = []
        for text in texts:
            response = self.client.embeddings.create(
                model=self.model,
                input=text,
            )
            embedding = response.data[0].embedding
            embeddings.append(embedding)

        return embeddings

    def embed_query(self, text: str) -> List[float]:
        """Embed a query using the Nebius API.

        Args:
            text: The text to embed

        Returns:
            Embedding for the text
        """
        response = self.client.embeddings.create(
            model=self.model,
            input=text,
        )
        return response.data[0].embedding

    async def aembed_documents(self, texts: List[str]) -> List[List[float]]:
        """Asynchronously embed a list of documents using the Nebius API.

        Note: This implementation falls back to the synchronous version as
        the Nebius OpenAI client might not have native async support.
        Consider implementing a fully async version if needed.

        Args:
            texts: The list of texts to embed

        Returns:
            List of embeddings, one for each text
        """
        return self.embed_documents(texts)

    async def aembed_query(self, text: str) -> List[float]:
        """Asynchronously embed a query using the Nebius API.

        Note: This implementation falls back to the synchronous version as
        the Nebius OpenAI client might not have native async support.
        Consider implementing a fully async version if needed.

        Args:
            text: The text to embed

        Returns:
            Embedding for the text
        """
        return self.embed_query(text)

In [32]:
# print("OpenAI components initialized")

import os

embeddings = NebiusE5MistralEmbeddings(
    api_key=os.getenv('NEBIUS_API_KEY'),  # Or set NEBIUS_API_KEY environment variable
    model="BAAI/bge-multilingual-gemma2"
)

# Define the LLM with Nebius AI Studio
llm = LLM(
    model="openai/meta-llama/Meta-Llama-3.1-70B-Instruct",
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv('NEBIUS_API_KEY')
)

# Setting Up the Couchbase Vector Store
A vector store is where we'll keep our embeddings. Unlike the FTS index, which is used for text-based search, the vector store is specifically designed to handle embeddings and perform similarity searches. When a user inputs a query, the search engine converts the query into an embedding and compares it against the embeddings stored in the vector store. This allows the engine to find documents that are semantically similar to the query, even if they don't contain the exact same words. By setting up the vector store in Couchbase, we create a powerful tool that enables our search engine to understand and retrieve information based on the meaning and context of the query, rather than just the specific words used.

In [33]:
# Setup vector store
vector_store = CouchbaseSearchVectorStore(
    cluster=cluster,
    bucket_name=CB_BUCKET_NAME,
    scope_name=SCOPE_NAME,
    collection_name=COLLECTION_NAME,
    embedding=embeddings,
    index_name=INDEX_NAME,
)
print("Vector store initialized")

Vector store initialized


# Load the BBC News Dataset
To build a search engine, we need data to search through. We use the BBC News dataset from RealTimeData, which provides real-world news articles. This dataset contains news articles from BBC covering various topics and time periods. Loading the dataset is a crucial step because it provides the raw material that our search engine will work with. The quality and diversity of the news articles make it an excellent choice for testing and refining our search engine, ensuring it can handle real-world news content effectively.

The BBC News dataset allows us to work with authentic news articles, enabling us to build and test a search engine that can effectively process and retrieve relevant news content. The dataset is loaded using the Hugging Face datasets library, specifically accessing the "RealTimeData/bbc_news_alltime" dataset with the "2024-12" version.

In [16]:
try:
    news_dataset = load_dataset(
        "RealTimeData/bbc_news_alltime", "2024-12", split="train"
    )
    print(f"Loaded the BBC News dataset with {len(news_dataset)} rows")
    logging.info(f"Successfully loaded the BBC News dataset with {len(news_dataset)} rows.")
except Exception as e:
    raise ValueError(f"Error loading the BBC News dataset: {str(e)}")

README.md:   0%|          | 0.00/55.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.48M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2687 [00:00<?, ? examples/s]

Loaded the BBC News dataset with 2687 rows


## Cleaning up the Data
We will use the content of the news articles for our RAG system.

The dataset contains a few duplicate records. We are removing them to avoid duplicate results in the retrieval stage of our RAG system.

In [17]:
news_articles = news_dataset["content"]
unique_articles = set()
for article in news_articles:
    if article:
        unique_articles.add(article)
unique_news_articles = list(unique_articles)
print(f"We have {len(unique_news_articles)} unique articles in our database.")

We have 1749 unique articles in our database.


## Saving Data to the Vector Store
To efficiently handle the large number of articles, we process them in batches of articles at a time. This batch processing approach helps manage memory usage and provides better control over the ingestion process.

We first filter out any articles that exceed 50,000 characters to avoid potential issues with token limits. Then, using the vector store's add_texts method, we add the filtered articles to our vector database. The batch_size parameter controls how many articles are processed in each iteration.

This approach offers several benefits:
1. Memory Efficiency: Processing in smaller batches prevents memory overload
2. Error Handling: If an error occurs, only the current batch is affected
3. Progress Tracking: Easier to monitor and track the ingestion progress
4. Resource Management: Better control over CPU and network resource utilization

We use a conservative batch size of 50 to ensure reliable operation.
The optimal batch size depends on many factors including:
- Document sizes being inserted
- Available system resources
- Network conditions
- Concurrent workload

Consider measuring performance with your specific workload before adjusting.


In [34]:
batch_size = 100

# Automatic Batch Processing
articles = [article for article in unique_news_articles if article and len(article) <= 50000]

try:
    vector_store.add_texts(
        texts=articles,
        batch_size=batch_size
    )
    logging.info("Document ingestion completed successfully.")
except Exception as e:
    raise ValueError(f"Failed to save documents to vector store: {str(e)}")

## Creating a Vector Search Tool
After loading our data into the vector store, we need to create a tool that can efficiently search through these vector embeddings. This involves two key components:

### Vector Retriever
The vector retriever is configured to perform similarity searches. This creates a retriever that performs semantic similarity searches against our vector database. The similarity search finds documents whose vector embeddings are closest to the query's embedding in the vector space.

### Search Tool
The search tool wraps the retriever in a user-friendly interface that:
- Takes a query string as input
- Passes the query to the retriever to find relevant documents
- Formats the results with clear document separation using document numbers and dividers
- Returns the formatted results as a single string with each document clearly delineated

The tool is designed to integrate seamlessly with our AI agents, providing them with reliable access to our knowledge base through vector similarity search. The lambda function in the tool handles both direct string queries and structured query objects, ensuring flexibility in how the tool can be invoked.


In [36]:
# Create vector retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
)

# Define the search tool using the @tool decorator
@tool("vector_search")
def search_tool(query: str) -> str:
    """Search for relevant documents using vector similarity.
    Input should be a simple text query string.
    Returns a list of relevant document contents.
    Use this tool to find detailed information about topics."""
    # Handle potential non-string query input if needed (similar to original lambda)
    # CrewAI usually passes the string directly based on task description
    # but checking doesn't hurt, though the Agent logic might handle this.
    # query_str = query if isinstance(query, str) else str(query.get('query', '')) # Simplified for now

    # Invoke the retriever
    docs = retriever.invoke(query)

    # Format the results
    formatted_docs = "\n\n".join([
        f"Document {i+1}:\n{'-'*40}\n{doc.page_content}"
        for i, doc in enumerate(docs)
    ])
    return formatted_docs

# Creating CrewAI Agents

We'll create two specialized AI agents using the CrewAI framework to handle different aspects of our information retrieval and analysis system:

## Research Expert Agent
This agent is designed to:
- Execute semantic searches using our vector store
- Analyze and evaluate search results
- Identify key information and insights
- Verify facts across multiple sources
- Synthesize findings into comprehensive research summaries

## Technical Writer Agent  
This agent is responsible for:
- Taking research findings and structuring them logically
- Converting technical concepts into clear explanations
- Ensuring proper citation and attribution
- Maintaining engaging yet informative tone
- Producing well-formatted final outputs

The agents work together in a coordinated way:
1. Research agent finds and analyzes relevant documents
2. Writer agent takes those findings and crafts polished responses
3. Both agents use a custom response template for consistent output

This multi-agent approach allows us to:
- Leverage specialized expertise for different tasks
- Maintain high quality through separation of concerns
- Create more comprehensive and reliable outputs
- Scale the system's capabilities efficiently

In [37]:
# Custom response template
response_template = """
Analysis Results
===============
{%- if .Response %}
{{ .Response }}
{%- endif %}

Sources
=======
{%- for tool in .Tools %}
* {{ tool.name }}
{%- endfor %}

Metadata
========
* Confidence: {{ .Confidence }}
* Analysis Time: {{ .ExecutionTime }}
"""

# Create research agent
researcher = Agent(

)

# Create writer agent
writer = Agent(

)

print("Agents created successfully")

Agents created successfully


## How CrewAI Agents Work in this RAG System

### Agent-Based RAG Architecture

This system uses a two-agent approach to implement Retrieval-Augmented Generation (RAG):

1. **Research Expert Agent**:
   - Receives the user query
   - Uses the vector search tool to retrieve relevant documents from Couchbase
   - Analyzes and synthesizes information from retrieved documents
   - Produces a comprehensive research summary with key findings

2. **Technical Writer Agent**:
   - Takes the research summary as input
   - Structures and formats the information
   - Creates a polished, user-friendly response
   - Ensures proper attribution and citation

#### How the Process Works:

1. **Query Processing**: User query is passed to the Research Agent
2. **Vector Search**: Query is converted to embeddings and matched against document vectors
3. **Document Retrieval**: Most similar documents are retrieved from Couchbase
4. **Analysis**: Research Agent analyzes documents for relevance and extracts key information
5. **Synthesis**: Research Agent combines findings into a coherent summary
6. **Refinement**: Writer Agent restructures and enhances the content
7. **Response Generation**: Final polished response is returned to the user

This multi-agent approach separates concerns (research vs. writing) and leverages
specialized expertise for each task, resulting in higher quality responses.


# Testing the Search System

Test the system with some example queries.

In [38]:
def process_query(query, researcher, writer):
    """
    Test the complete RAG system with a user query.

    This function tests both the vector search capability and the agent-based processing:
    1. Vector search: Retrieves relevant documents from Couchbase
    2. Agent processing: Uses CrewAI agents to analyze and format the response

    The function measures performance and displays detailed outputs from each step.
    """
    print(f"\nQuery: {query}")
    print("-" * 80)

    # Create tasks
    research_task = Task(

    )

    writing_task = Task(

    )

    # Create and execute crew
    crew = Crew(
        agents=[researcher, writer],
        tasks=[research_task, writing_task],
        process=Process.sequential,
        verbose=True,
        cache=True,
        planning=True
    )

    try:
        start_time = time.time()
        result = crew.kickoff()
        elapsed_time = time.time() - start_time

        print(f"\nQuery completed in {elapsed_time:.2f} seconds")
        print("=" * 80)
        print("RESPONSE")
        print("=" * 80)
        print(result)

        if hasattr(result, 'tasks_output'):
            print("\n" + "=" * 80)
            print("DETAILED TASK OUTPUTS")
            print("=" * 80)
            for task_output in result.tasks_output:
                print(f"\nTask: {task_output.description[:100]}...")
                print("-" * 40)
                print(f"Output: {task_output.raw}")
                print("-" * 40)
    except Exception as e:
        print(f"Error executing crew: {str(e)}")
        logging.error(f"Crew execution failed: {str(e)}", exc_info=True)

In [42]:
# Disable logging before running the query
logging.disable(logging.CRITICAL)

import os

query = "What are the key details about the FA Cup third round draw? Include information about Manchester United vs Arsenal, Tamworth vs Tottenham, and other notable fixtures"
process_query(query, researcher, writer)


Query: What are the key details about the FA Cup third round draw? Include information about Manchester United vs Arsenal, Tamworth vs Tottenham, and other notable fixtures
--------------------------------------------------------------------------------


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5942708d-d258-4fbb-b479-c00cf04d5c5d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 
[2025-06-17 04:21:22][INFO]: Planning the crew execution


🚀 Crew: crew
└── 📋 Task: 924e943e-4d68-4598-8d0e-2664fd4c6687
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 924e943e-4d68-4598-8d0e-2664fd4c6687
       Status: Executing Task...
    └── 🤖 Agent: Task Execution Planner
            Status: In Progress

...

🚀 Crew: crew
├── 📋 Task: 924e943e-4d68-4598-8d0e-2664fd4c6687
│      Assigned to: Task Execution Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Task Execution Planner
│           Status: ✅ Completed
├── 📋 Task: 1463281c-4368-4e8a-8556-8885ae697f36
│      Assigned to: Research Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Expert
│           Status: ✅ Completed
└── 📋 Task: b66211c0-5577-4d0e-84e8-f7ce48ea09b6
       Status: Executing Task...
    └── 🤖 Agent: Technical Writer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 924e943e-4d68-4598-8d0e-2664fd4c6687
│      Assigned to: Task Execution Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Task Execution Planner
│           Status: ✅ Completed
├── 📋 Task: 1463281c-4368-4e8a-8556-8885ae697f36
│      Assigned to: Research Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Expert
│           Status: ✅ Completed
└── 📋 Task: b66211c0-5577-4d0e-84e8-f7ce48ea09b6
       Assigned to: Technical Writer
       Status: ✅ Completed
    └── 🤖 Agent: Technical Writer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b66211c0-5577-4d0e-84e8-f7ce48ea09b6                                                                     │
│  Agent: Technical Writer                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5942708d-d258-4fbb-b479-c00cf04d5c5d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Query completed in 62.16 seconds
RESPONSE
Comprehensive Response: FA Cup Third Round Draw Analysis

The Football Association Challenge Cup, commonly known as the FA Cup, is the oldest association football competition in the world. The tournament is open to all clubs playing in the top 10 levels of the English football league system, making it a true test of endurance and skill for teams of all calibers. The third round draw has been announced, and it promises to be an exciting weekend of football, with several intriguing fixtures lined up.

**Introduction to the FA Cup**

The FA Cup is steeped in history and tradition, with the first tournament held in 1871-72. The competition has grown in popularity over the years, with teams from non-league and lower league divisions having the opportunity to compete against top-flight Premier League teams. The FA Cup is known for its giant-killing acts, where smaller teams defeat more established opponents, adding to the excitement and unpredictabi

## Conclusion
By following these steps, you've built a powerful RAG system that combines Couchbase's vector storage capabilities with CrewAI's agent-based architecture. This multi-agent approach separates research and writing concerns, resulting in higher quality responses to user queries.

The system demonstrates several key advantages:
1. Efficient vector search using Couchbase's vector store
2. Specialized AI agents that focus on different aspects of the RAG pipeline
3. Collaborative workflow between agents to produce comprehensive, well-structured responses
4. Scalable architecture that can be extended with additional agents for more complex tasks

Whether you're building a customer support system, a research assistant, or a knowledge management solution, this agent-based RAG approach provides a flexible foundation that can be adapted to various use cases and domains.